# 📱 PocketAI APK Builder

This notebook compiles the PocketAI Expo project into an APK from GitHub.

**Repository:** [mithun50/PocketAi-Expo](https://github.com/mithun50/PocketAi-Expo)

## Steps:
1. Setup Build Tools (Java, Ninja, Android SDK)
2. Install Node.js & Dependencies
3. Clone from GitHub
4. Build APK with Ninja
5. Download APK

---

**Just click `Runtime > Run all` to build!**

## 1️⃣ Install Java JDK 17 & Ninja Build

In [ ]:
%%bash
echo "📦 Installing Java JDK 17 and Ninja..."
apt-get update -qq
apt-get install -qq -y openjdk-17-jdk-headless ninja-build cmake > /dev/null 2>&1
echo "✅ Java installed: $(java -version 2>&1 | head -1)"
echo "✅ Ninja installed: $(ninja --version)"
echo "✅ CMake installed: $(cmake --version | head -1)"

## 2️⃣ Install Android SDK, NDK & Build Tools

In [ ]:
%%bash
echo "📦 Installing Android SDK with NDK..."

# Set up directories
export ANDROID_HOME=/root/android-sdk
export ANDROID_SDK_ROOT=$ANDROID_HOME
mkdir -p $ANDROID_HOME/cmdline-tools

# Download command-line tools
cd /tmp
wget -q https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip -O cmdline-tools.zip
unzip -q -o cmdline-tools.zip
mv cmdline-tools $ANDROID_HOME/cmdline-tools/latest

# Set PATH
export PATH=$PATH:$ANDROID_HOME/cmdline-tools/latest/bin:$ANDROID_HOME/platform-tools

# Accept licenses and install components including NDK
yes | sdkmanager --licenses > /dev/null 2>&1
sdkmanager --install \
  "platform-tools" \
  "platforms;android-34" \
  "build-tools;34.0.0" \
  "ndk;26.1.10909125" \
  "cmake;3.22.1" > /dev/null 2>&1

echo "✅ Android SDK installed"
echo "   Build Tools: 34.0.0"
echo "   Platform: android-34"
echo "   NDK: 26.1.10909125"
echo "   CMake: 3.22.1"

In [ ]:
# Set environment variables for Python cells
import os

os.environ['ANDROID_HOME'] = '/root/android-sdk'
os.environ['ANDROID_SDK_ROOT'] = '/root/android-sdk'
os.environ['ANDROID_NDK_HOME'] = '/root/android-sdk/ndk/26.1.10909125'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH'] = os.environ['PATH'] + ':/root/android-sdk/cmdline-tools/latest/bin:/root/android-sdk/platform-tools'

print("✅ Environment variables configured")
print(f"   ANDROID_HOME: {os.environ['ANDROID_HOME']}")
print(f"   ANDROID_NDK_HOME: {os.environ['ANDROID_NDK_HOME']}")

## 3️⃣ Install Node.js 20 LTS

In [ ]:
%%bash
echo "📦 Installing Node.js 20 LTS..."

# Install Node.js using NodeSource
curl -fsSL https://deb.nodesource.com/setup_20.x | bash - > /dev/null 2>&1
apt-get install -qq -y nodejs > /dev/null 2>&1

echo "✅ Node.js installed: $(node -v)"
echo "✅ npm installed: $(npm -v)"

## 4️⃣ Clone from GitHub

In [ ]:
#@title GitHub Repository Settings
#@markdown Configure your repository:

github_repo = "https://github.com/mithun50/PocketAi-Expo.git" #@param {type:"string"}
github_branch = "main" #@param {type:"string"}

import os
import shutil

PROJECT_DIR = "/content/pocketai-expo"

# Clean previous builds
if os.path.exists(PROJECT_DIR):
    shutil.rmtree(PROJECT_DIR)
    print("🧹 Cleaned previous build")

print(f"")
print(f"📥 Cloning repository...")
print(f"   Repo: {github_repo}")
print(f"   Branch: {github_branch}")
print(f"")

!git clone --depth 1 -b {github_branch} {github_repo} {PROJECT_DIR}

print(f"")
print(f"✅ Repository cloned to {PROJECT_DIR}")

## 5️⃣ Install Dependencies & Prebuild

In [ ]:
%%bash
cd /content/pocketai-expo

echo "📦 Installing npm dependencies..."
npm install --legacy-peer-deps 2>&1 | tail -5

echo ""
echo "🔧 Running Expo prebuild..."
npx expo prebuild --platform android --clean 2>&1 | tail -10

echo ""
echo "✅ Prebuild complete!"

## 6️⃣ Configure Ninja Build

In [ ]:
%%bash
cd /content/pocketai-expo/android

echo "⚡ Configuring Ninja build for faster compilation..."

# Create or update local.properties with NDK path
cat > local.properties << EOF
sdk.dir=/root/android-sdk
ndk.dir=/root/android-sdk/ndk/26.1.10909125
EOF

# Add ninja configuration to gradle.properties
cat >> gradle.properties << EOF

# Ninja build configuration
android.useAndroidX=true
android.enableJetifier=true
org.gradle.jvmargs=-Xmx4g -XX:MaxMetaspaceSize=512m -XX:+HeapDumpOnOutOfMemoryError
org.gradle.parallel=true
org.gradle.caching=true
org.gradle.configureondemand=true
android.native.buildOutput=verbose
EOF

echo "✅ Ninja build configured"
echo "   Parallel builds: enabled"
echo "   Gradle caching: enabled"
echo "   Max heap: 4GB"

## 7️⃣ Build APK with Ninja

In [ ]:
#@title Build Configuration
#@markdown Select build type:

build_type = "release" #@param ["debug", "release"]

import os
os.chdir("/content/pocketai-expo/android")

if build_type == "debug":
    gradle_task = "assembleDebug"
    apk_path = "/content/pocketai-expo/android/app/build/outputs/apk/debug/app-debug.apk"
else:
    gradle_task = "assembleRelease"
    apk_path = "/content/pocketai-expo/android/app/build/outputs/apk/release/app-release.apk"

print(f"🔨 Building {build_type} APK with Ninja...")
print(f"   Task: {gradle_task}")
print(f"   Output: {apk_path}")

In [ ]:
%%bash
cd /content/pocketai-expo/android

# Set environment
export ANDROID_HOME=/root/android-sdk
export ANDROID_SDK_ROOT=/root/android-sdk
export ANDROID_NDK_HOME=/root/android-sdk/ndk/26.1.10909125
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export PATH=$PATH:/root/android-sdk/cmake/3.22.1/bin

echo "🔨 Building APK with Ninja (this may take 5-10 minutes)..."
echo "⚡ Using parallel builds and Gradle caching for speed"
echo ""

# Make gradlew executable
chmod +x ./gradlew

# Build with optimizations
./gradlew assembleRelease \
  --no-daemon \
  --warning-mode=none \
  --parallel \
  --max-workers=4 \
  -Dorg.gradle.parallel=true \
  -Dorg.gradle.caching=true \
  2>&1 | grep -E '(BUILD|FAILED|ERROR|Task|%|:app:|SUCCESS)'

echo ""
if [ -f "app/build/outputs/apk/release/app-release.apk" ]; then
    echo "✅ Release APK built successfully!"
    ls -lh app/build/outputs/apk/release/app-release.apk
elif [ -f "app/build/outputs/apk/debug/app-debug.apk" ]; then
    echo "✅ Debug APK built successfully!"
    ls -lh app/build/outputs/apk/debug/app-debug.apk
else
    echo "❌ Release build failed. Trying debug build..."
    ./gradlew assembleDebug --no-daemon --parallel 2>&1 | tail -20
fi

## 8️⃣ Download APK

In [ ]:
from google.colab import files
import os
import shutil

# Find the APK
apk_paths = [
    "/content/pocketai-expo/android/app/build/outputs/apk/release/app-release.apk",
    "/content/pocketai-expo/android/app/build/outputs/apk/debug/app-debug.apk"
]

apk_found = None
for path in apk_paths:
    if os.path.exists(path):
        apk_found = path
        break

if apk_found:
    # Rename to something meaningful
    final_name = "PocketAI.apk"
    final_path = f"/content/{final_name}"
    shutil.copy(apk_found, final_path)

    size_mb = os.path.getsize(final_path) / (1024 * 1024)
    print(f"📱 APK Ready: {final_name}")
    print(f"📦 Size: {size_mb:.2f} MB")
    print(f"")
    print(f"⬇️ Starting download...")
    files.download(final_path)
else:
    print("❌ No APK found. Check build logs above.")
    print("")
    print("Available files in build output:")
    !find /content/pocketai-expo/android/app/build/outputs -name "*.apk" 2>/dev/null || echo "No APK files found"

---

## 🔧 Troubleshooting

### Common Issues:

**Build fails with SDK error:**
- Re-run cell 2 (Android SDK installation)

**Out of memory:**
- Go to `Runtime > Change runtime type > High-RAM`

**Gradle daemon issues:**
- `Runtime > Restart runtime` and run all cells again

**Clone failed:**
- Check if repository is public
- Verify branch name exists

**Native build failed:**
- Check NDK installation in cell 2
- Verify ninja is installed in cell 1

In [ ]:
#@title 🔍 Debug: Check Build Environment

!echo "=== Java ==="
!java -version 2>&1 | head -1
!echo ""
!echo "=== Ninja ==="
!ninja --version
!echo ""
!echo "=== Node.js ==="
!node -v
!echo ""
!echo "=== Android SDK ==="
!ls -la $ANDROID_HOME/build-tools/ 2>/dev/null || echo "Not found"
!echo ""
!echo "=== Android NDK ==="
!ls -la $ANDROID_HOME/ndk/ 2>/dev/null || echo "Not found"
!echo ""
!echo "=== Project Structure ==="
!ls -la /content/pocketai-expo/ 2>/dev/null | head -15 || echo "Project not found"